In [1]:
import os
from tqdm.notebook import tqdm
import gzip
import pandas as pd
import seaborn as sns
import timeit
import json
import matplotlib.pyplot as plt
import numpy as np
import emoji


# Tweets

In [2]:
tweets = pd.read_pickle('data/tweets.pkl.bz2', compression='bz2')
tweets.head()

id          created_at              source  \
0  1588682493833551872 2022-11-04 23:59:57  Twitter for iPhone   
1  1588682474141339648 2022-11-04 23:59:52     Twitter Web App   
2  1588682395393622016 2022-11-04 23:59:34  Twitter for iPhone   
3  1588682374103334912 2022-11-04 23:59:28  Twitter for iPhone   
4  1588682360253739008 2022-11-04 23:59:25  Twitter for iPhone   

   possibly_sensitive            author_id  \
0                   0            580633644   
1                   1           2711676037   
2                   0  1573331383984226304   
3                   0           1674604256   
4                   0  1144336826133442560   

                                                text  person_annotations  \
0  The business of businesses is climate-change a...                   0   
1  #howtowrite #howtopublish @authoraid \nA case ...                   0   
2  Hundreds of elephants, zebras die as Kenya wea...                   0   
3  If the free market is the answer to our climat...                   0   
4  Tuned in to the local Seattle fake, corrupt, w...                   0   

   place_annotations  product_annotations  organization_annotations  ...  \
0                  0                    0                         0  ...   
1                  0                    0                         0  ...   
2                  0                    0                         0  ...   
3                  1                    0                         0  ...   
4                  1                    0                         0  ...   

   like_count  quote_count                      author_name  verified  \
0           0            0        Robert Han @RobertHon0911         0   
1           1            0   Krishna Koushik @SapiensMyopia         0   
2           1            0      Nature Chick @EarthBulletin         0   
3           0            0       Nick Yates @nickyatesworld         0   
4           3            0  greg vanommeren @GregVanommeren         0   

   protected  profile_image_url  author_followers_count  animated_gif_count  \
0          0                  1                      57                   0   
1          0                  1                      56                   0   
2          0                  1                     553                   0   
3          0                  1                     190                   0   
4          0                  1                     521                   0   

   photo_count  video_count  
0            0            0  
1            0            0  
2            0            0  
3            0            0  
4            0            0  

[5 rows x 30 columns]

In [3]:
tweets.shape

(7581574, 30)

In [4]:
tweets.drop_duplicates(subset=['id'], inplace=True)
tweets.shape

(7581574, 30)

In [8]:
# remove emojis
import re
def find_emojis(text):
        regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
        emojis = regrex_pattern.search(text)
        return emojis


indexs = tweets.text[~tweets['text'].map(find_emojis).isna()].index
indexs

Int64Index([  12988,   40180,   92294,  120404,  139908,  194635,  290973,
             347163,  441678,  461956,
            ...
            7036685, 7036976, 7037268, 7037428, 7155838, 7174035, 7302006,
            7396346, 7457861, 7458562],
           dtype='int64', length=159)

In [6]:
from emoji_translate.emoji_translate import Translator

emo = Translator(exact_match_only=False, randomize=True)

# basic translation
print(emo.demojify('The 🏠 is on 🔥!'))         # > 'The house is on fire!'

The house is on fire!


In [9]:
tweets.loc[indexs,"text"] = tweets.loc[indexs,"text"].map(emo.demojify)
tweets.iloc[12988]

id                                                        1588379209810206720
created_at                                                2022-11-04 03:54:48
source                                                    Twitter for Android
possibly_sensitive                                                          0
author_id                                                 1393778718032764928
text                        ONE HEALTH connects people :man_farmer:woman‍a...
person_annotations                                                          0
place_annotations                                                           0
product_annotations                                                         0
organization_annotations                                                    0
other_annotations                                                           0
urls_count                                                                  1
url_image_count                                                 

In [10]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7581574 entries, 0 to 7581709
Data columns (total 30 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   id                        int64         
 1   created_at                datetime64[ns]
 2   source                    object        
 3   possibly_sensitive        int64         
 4   author_id                 int64         
 5   text                      object        
 6   person_annotations        int64         
 7   place_annotations         int64         
 8   product_annotations       int64         
 9   organization_annotations  int64         
 10  other_annotations         int64         
 11  urls_count                int64         
 12  url_image_count           int64         
 13  mentions_count            int64         
 14  mentions_verifications    int64         
 15  mentions_followers        int64         
 16  hashtags                  int64         
 17  cashtags

In [11]:
tweets.to_pickle('data/tweets.pkl.bz2',  compression='bz2')

In [ ]:
data = pd.read_pickle('../tweets.pkl.bz2', compression='bz2')
data.info()

# Paired tweets

We should drop the text columns as they are memory intensive and unnecessary as we have already embedded the text for each tweet.